In [1]:
# setting notebook source
with open('./config_path.py') as f:
    exec(f.read())

In [2]:
from config import secrets, settings
from zoho_api import *
import pandas as pd
import copy

In [ ]:
backup_data = {}
response_data = {}

In [ ]:
refresh_token()

In [ ]:
SLAs = {}
SLAs['SupportME Premium'] = '10977000006774075'
SLAs['SupportME Comfort'] = '10977000006804083'
SLAs['SupportME Standard'] = '10977000006804131'
SLAs['SupportME StartUpCare'] = '10977000000007203'
SLAs['Default'] = '10977000000007201'
SLAs

## App

In [ ]:
# Current data from Zoho
data_source = r"C:\Users\KutaP\OneDrive - Mitsubishi Electric Europe B.V\Zoho\Dane Raportowe\Accounts_analysis.xlsx"
# Data to be updated
data_update = r"C:\Users\KutaP\Desktop\Serwisy - Podstawowy (1).xlsx"

In [ ]:
# READ ZOHO ID AS STRING
stg_df = pd.read_excel(data_source, sheet_name='Merge SAP Zoho', dtype= {'Account Reference Id': object} )

In [ ]:
df = copy.deepcopy(stg_df)
#filtering
df = df[df['Zoho SAP_ID_Integer']>0] 

In [ ]:
# data to update
update_df = pd.read_excel(data_update, sheet_name='Values', index_col='SAP ID Klienta')

In [ ]:
# joining data to have Zoho Account Reference Id and required new information in one table
joined_df = update_df.join(df.set_index('ERP Account ID'))

In [ ]:
columns_to_keep = ['Zoho SAP ID',
       'Account Reference Id', 'Klient', 'Nazwa usługi', 'Data rozpoczęcia usługi',
       'Data zakończenia usługi', 'Opiekun klienta', 'Program Lojalnościowy']

In [ ]:
joined_df = joined_df[columns_to_keep].dropna(subset =['Account Reference Id']).set_index('Account Reference Id')

In [ ]:
joined_df['Nazwa usługi'] = joined_df['Nazwa usługi'].apply(lambda x: " ".join(x.split(' ')[:2]))

In [ ]:
joined_df['SLA'] = joined_df['Nazwa usługi'].apply(lambda x: SLAs[x])

In [ ]:
for record in joined_df.iterrows():
    accountId = record[0]
    account = backup_updated_fields(accountId, {'associatedSLAIds': None}, backup_data, response_data)
    apply_sla(account_id=accountId, sla_id=record[1]['SLA'] )